<a href="https://colab.research.google.com/github/graviada/DataScienceProjects/blob/main/PetProjects/Toxic_Comments_BinaryClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Задача бизнеса:**
    Необходимо разработать модель, которая будет удалять негативные комментарии на форуме.
### **Постановка задачи:**
    - разработать бинарный классификатор (воспользуемся логистической регрессией);
    - максимизировать метрику Recall;
    - добиться Precision > 0.95.

Так как мы не обладаем размеченной выборкой для обучения модели, возможно использование готового датасета с негативными и позитивными комментариями на русском языке.

Для реализации модели был выбран датасет Russian Language Toxic Comments Dataset.

Он представляет собой коллекцию аннотированных комментариев с сайтов Двач и Пикабу. Опубликован на Kaggle в 2019-м и содержит 14 412 комментариев, из которых 4 826 помечены как токсичные, а 9 586 — как нетоксичные. Средняя длина комментария 175 символов, минимальная — 21, максимальная — 7 403.

### **Импорт библиотек**

In [11]:
import pandas as pd
from google.colab import files

### **Загрузка датасета с Kaggle**

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"irinachalova","key":"3d68b18a63cbb1fb592d9e53facb0454"}'}

In [4]:
# Загружаем API-клиента Kaggle
!pip install -q kaggle

In [5]:
# Kaggle API-клиент ожидает, что мы сохраним скаченный файл в ~/.kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Это изменение разрешений позволяет избежать предупреждения при запуске инструмента Kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
# Отобразим доступные датасеты
!kaggle datasets list

ref                                                            title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
datasets/muratkokludataset/date-fruit-datasets                 Date Fruit Datasets                              408KB  2022-04-03 09:25:39            686        196  0.9375           
datasets/piterfm/2022-ukraine-russian-war                      2022 Ukraine Russia War                            2KB  2022-04-16 14:04:28           8601        481  1.0              
datasets/muratkokludataset/acoustic-extinguisher-fire-dataset  Acoustic Extinguisher Fire Dataset               621KB  2022-04-02 22:59:36             34        179  0.9375           
datasets/kamilpytlak/personal-key-indicators-of-heart-disease  Personal Key Indi

In [21]:
!kaggle datasets download -d blackmoon/russian-language-toxic-comments

russian-language-toxic-comments.zip: Skipping, found more recently modified local copy (use --force to force download)


In [22]:
!ls

kaggle.json  russian-language-toxic-comments.zip  sample_data


In [23]:
!unzip russian-language-toxic-comments.zip

Archive:  russian-language-toxic-comments.zip
  inflating: labeled.csv             


In [24]:
!ls

kaggle.json  labeled.csv  russian-language-toxic-comments.zip  sample_data


### **Разведочный анализ данных (англ. exploratory data analysis, EDA)**

In [27]:
df = pd.read_csv('labeled.csv')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comment  14412 non-null  object 
 1   toxic    14412 non-null  float64
dtypes: float64(1), object(1)
memory usage: 225.3+ KB
